# How to Write a Signal Processor

Ian Guinn, UNC.  Presented at [LEGEND Software Tutorial, Nov. 2021](https://indico.legend-exp.org/event/561/)

This tutorial will demonstrate, using several examples, how to write a signal processor. Signal processors are typically written using numba, a python library that generates high performance C-code from python. In particular, we will use the guvectorize command to write generalized ufuncs: https://numpy.org/doc/stable/reference/c-api/generalized-ufuncs.html

Compared to a typical python function, generalized ufuncs are required to specify more information. In particular, we must provide information about the data types of the inputs, the shape of the arrays that will be provided, and some other details about how the function will be generated. We will generate a few simple examples to demonstrate these concepts; in addition, a list of processors are provided in pygama/dsp/\_processors, providing even more examples. When you write an actual processor, you will add a new python module to this directory with the function defined, and then register it in pygama/dsp/processors.py, so that pygama has an easy time finding it.

In [ ]:
import numpy as np
from numba import guvectorize
from pygama.dsp.errors import DSPFatal

## Example 1: A simple processor

For our first, very simple example, we will produce a processor that adds a value to each element of a waveform. The function itself is written in normal looking python. As written, however, this would be a low performing function due to the for loop that it uses (python loops are slow; this is why we like vectorized functions). By compiling this function, numba will convert the loop to C, which will accelerate it considerably.

It is also important to notice that the output of the function is not returned, but instead is an argument of the function (w_out). While this is somewhat unusual in python, this is called "passing by reference" and is common practice in C and other languages; instead of creating a new array and returning it, this function will take a pre-defined memory block and fill it. This further aids the performance of the DSP by avoiding repeated allocation and deallocation of arrays, which can be a slow process. This function can still be called by passing only inputs; in that case the numbafied function will automatically "broadcast" the shape and type of the output array from the input values, and create it.

Prior to the definition of the function, we add the @guvectorize operator, with the following arguments:
1) A list of type signatures. Each signature will have one type for each function argument. The types are all primitive C-types, and can largely be described by a type (floating point, integer, unsigned integer) and a size (in bits). For this example, we are using 32-bit and 64-bit floats (aka doubles). For types that represent arrays and for outputs, we must also append [:]. If you are familiar with C, this can be thought of as passing a pointer
2) A shape-signature, with a parenthesis-enclosed shape variable for each argument. Scalar values are represented as (), 1D arrays by (n), 2D arrays by (n,m), etc. For arrays with the same character, each argument must be the same length (in this example, this means the output w_out must be the same size as w_in). Inputs and outputs are separated by ->; however, inputs can still be modified and treated as outputs. This becomes necessary when the output has a shape different from all inputs!
3) nopython: if set to True, this will force numba to compile C-code, or fail if it is unable to. Alternatively, one can use forceobj mode, which does not actually compile C-code; we will have an example of this soon.
4) cache: if set to True, after compiling the function, we will store it on disk. This avoids the need to recompile the function each time we restart python

In [ ]:
@guvectorize(["void(float32[:], float32, float32[:])",
              "void(float64[:], float64, float64[:])"],
             "(n),()->(n)", nopython=True, cache=True)
def test(a, b, c):
    for i, x in enumerate(a):
        c[i] = x + b

w_in = np.arange(10)
print("test("+str(np.arange(10))+", 3.) = "+str(test(w_in, 3.)))

## Example 2: Wrapping a python function

Sometimes, we want to be able to run python code from other scientific computing libraries such as scipy. It's cases like this, where we use forceobj instead of nopython mode. For this example, we will use scipy's fast fourier transform to compute.

In [ ]:
import scipy.fft

@guvectorize(["void(float32[:], complex64[:])",
              "void(float64[:], complex128[:])"],
             "(n)->(n)", forceobj=True)
def fft(w_in, fft_out):
    fft_out[:] = scipy.fft.fft(w_in)

w_in = np.arange(10)
print("fft(" + str(w_in) + ") = " + str(fft(np.arange(10))))


## Example 3: Pygama Style Specifications

For pygama, we have defined additional style specifications, which are laid out in pygama/dsp/\_processors/template.py. This defines naming schemes and which types to use. It also lays out how to handle exceptions and bounds checking using not-a-numbers (NaNs). Finally, it describes how to document the code using doc-strings.

We will revisit Example 1, applying these guidelines.

In [ ]:
@guvectorize(["void(float32[:], float32, float32[:])",
              "void(float64[:], float64, float64[:])"],
             "(n),()->(n)", nopython=True, cache=True)
def test(w_in, a_in, w_out):
    """This is a test function to demonstrate the basics of writing DSP processors. It adds a constant value to the input waveform.
    Parameters
    ----------
    w_in : input waveform
    a_in : input scalar added to input waveform
    w_out : output waveform
    """
    # If scalar is not finite, raise an exception
    if not np.isfinite(a_in):
        raise DSPFatal("Input scalar must not be NaN or inf")
    
    # If any NaNs appear in w_in, set w_out to all NaN
    if np.isnan(w_in).any():
        w_out[:] = np.nan
        return
    
    for i, x in enumerate(w_in):
        w_out[i] = x + a_in
